In [286]:
import numpy as np

In [287]:
class ANN:
    def __init__(self, layer_size= [50,1], lr= 0.001, momentum= 0, batch_size= None, init_seed = None, nb_epoch= 300):
        self.x = None
        self.y = None
        self.lr = lr
        self.momentum = momentum
        self.layer_size = layer_size
        self.weights = []
        self.biases = []
        self.velocity = []
        self.output_layer = []
        self.batch_size = batch_size
        
        self.nb_epoch = nb_epoch
        
        if init_seed != None:
            np.random.seed(init_seed)
            
        for i, layer in enumerate(layer_size):
            if i < (len(layer_size) - 1):
                self.weights.append(np.random.rand(layer_size[i], layer_size[i+1]))
                self.velocity.append(np.zeros((layer_size[i], layer_size[i+1])))
            self.biases.append(np.random.rand(layer_size[i]))
        
    def _sigmoid(self, x, derivative= False):
        sigmoid =  1 / (1 + np.exp(-x))
        if derivative:
            return sigmoid * (1 - sigmoid)
        else:
            return sigmoid
    
    def fit(self, x, y):
        self.x = np.array(x, dtype= np.float32)
        self.y = np.array(y, dtype= np.float32)
        if len(self.y.shape) == 1:
            self.y = np.array([[y] for y in self.y], dtype= np.float32)
        
        self.weights = [np.random.rand(self.x.shape[1], self.layer_size[0])] + self.weights
        self.velocity = [np.zeros((self.x.shape[1], self.layer_size[0]))] + self.velocity
        if self.batch_size is None:
            self.batch_size = self.x.shape[0]
        
        for i in range(self.nb_epoch):
            loss = 0
            indices = np.arange(self.y.shape[0])
            np.random.shuffle(indices)            
            self.x = self.x[indices]
            self.y = self.y[indices]
            
            for idx in range(0, self.x.shape[0], self.batch_size):
                data_in = self.x[idx : min(idx + self.batch_size, self.x.shape[0])]
                data_target = self.y[idx : min(idx + self.batch_size, self.x.shape[0])]
                out = self._feed_forward(data_in)
                loss += np.sum((out-data_target)**2)
                self._backprop(data_in, data_target)
            print("Epoch {}, loss: {}".format(i, loss))
    
    def _feed_forward(self, input_data= []):
        self.output_layer = []
        in_layer = input_data
        for weight, bias in zip(self.weights, self.biases):
            out_layer = self._sigmoid(np.dot(in_layer, weight) + bias)
            self.output_layer.append(out_layer)
            
            in_layer = out_layer
        return out_layer
        
    def _backprop(self, data_in, data_target):
        for i, w in reversed(list(enumerate(self.weights))):
            if i == len(self.weights) - 1:
                out_d = 2 * (data_target - self.output_layer[i]) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = np.dot(self.output_layer[i - 1].T, out_d) + self.velocity[i] * self.momentum
            elif i == 0:
                out_d = np.dot(out_d, self.weights[i + 1].T) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = np.dot(data_in.T, out_d) + self.velocity[i] * self.momentum
            else:
                out_d = np.dot(out_d, self.weights[i + 1].T) * self._sigmoid(self.output_layer[i], derivative= True)
                d_weight = np.dot(self.output_layer[i - 1].T, out_d) + self.velocity[i] * self.momentum
            
            self.velocity[i] = d_weight
        
        for i, w in enumerate(self.weights):
            self.weights[i] += self.velocity[i]
            
    
    def predict(self, data):
        data = np.array(data, dtype= np.float32)
        return self._feed_forward(data)
    
    def get_weights(self):
        return self.weights
    
    def get_biases(self):
        return self.biases

## Experiment 1
Data point

In [301]:
ann = ANN(layer_size=[4, 4, 1], momentum= 0.005, lr= 0.01, batch_size= 4, nb_epoch= 150)
x = [[0,1], [0,0], [1,0], [1,1]]
y = [[0], [0], [1], [1]]


ann.fit(x, y)
ann.predict([[0,0], [0,0.1]])

Epoch 0, loss: 1.6069532481795483
Epoch 1, loss: 1.1425182415577453
Epoch 2, loss: 1.0150783086451496
Epoch 3, loss: 1.0112568846179064
Epoch 4, loss: 1.0107702982449325
Epoch 5, loss: 1.010294459933176
Epoch 6, loss: 1.0098214653353639
Epoch 7, loss: 1.0093511330317906
Epoch 8, loss: 1.0088831224084278
Epoch 9, loss: 1.0084170542893125
Epoch 10, loss: 1.0079525379781344
Epoch 11, loss: 1.0074891709949214
Epoch 12, loss: 1.0070265377451246
Epoch 13, loss: 1.0065642082072441
Epoch 14, loss: 1.0061017365768576
Epoch 15, loss: 1.005638659848557
Epoch 16, loss: 1.005174496324622
Epoch 17, loss: 1.0047087440389106
Epoch 18, loss: 1.0042408790836173
Epoch 19, loss: 1.003770353825593
Epoch 20, loss: 1.0032965949977795
Epoch 21, loss: 1.0028190016499998
Epoch 22, loss: 1.0023369429418116
Epoch 23, loss: 1.0018497557584007
Epoch 24, loss: 1.0013567421284653
Epoch 25, loss: 1.000857166420769
Epoch 26, loss: 1.000350252293416
Epoch 27, loss: 0.9998351793669309
Epoch 28, loss: 0.999311079588819
Ep

array([[0.02588839],
       [0.02736542]])

## Experiment 2
Data weather

In [289]:
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Loading data
data, meta = arff.loadarff('weather.arff')
data_x = [list(d)[:4] for d in data]
data_y = [[d[4]] for d in data]

# encoding data
le = LabelEncoder()
data_y = le.fit_transform(data_y)

ct = ColumnTransformer([('ohe', OneHotEncoder(), [0, 3]),], remainder= 'passthrough')
data_x = ct.fit_transform(data_x)

# splitting data into train and test data
train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size= 0.1)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [306]:
ann = ANN(layer_size=[64, 1], momentum= 0, lr= 0.025, nb_epoch= 300, batch_size= 8)
ann.fit(train_x, train_y)
pred = ann.predict(test_x)

print("Prediction: {}".format(pred))
print("Target: {}".format(test_y))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in exp


Epoch 0, loss: 4.577117681769888
Epoch 1, loss: 2.77523638612105
Epoch 2, loss: 2.77523638612105
Epoch 3, loss: 2.77523638612105
Epoch 4, loss: 2.77523638612105
Epoch 5, loss: 2.77523638612105
Epoch 6, loss: 2.77523638612105
Epoch 7, loss: 2.77523638612105
Epoch 8, loss: 2.77523638612105
Epoch 9, loss: 2.77523638612105
Epoch 10, loss: 2.77523638612105
Epoch 11, loss: 2.77523638612105
Epoch 12, loss: 2.77523638612105
Epoch 13, loss: 2.77523638612105
Epoch 14, loss: 2.77523638612105
Epoch 15, loss: 2.77523638612105
Epoch 16, loss: 2.77523638612105
Epoch 17, loss: 2.77523638612105
Epoch 18, loss: 2.77523638612105
Epoch 19, loss: 2.77523638612105
Epoch 20, loss: 2.77523638612105
Epoch 21, loss: 2.77523638612105
Epoch 22, loss: 2.77523638612105
Epoch 23, loss: 2.77523638612105
Epoch 24, loss: 2.77523638612105
Epoch 25, loss: 2.77523638612105
Epoch 26, loss: 2.77523638612105
Epoch 27, loss: 2.77523638612105
Epoch 28, loss: 2.77523638612105
Epoch 29, loss: 2.77523638612105
Epoch 30, loss: 2.7

Epoch 299, loss: 2.77523638612105
Prediction: [[0.57154844]
 [0.57154844]]
Target: [1 0]
